In [174]:
# one euro filter class definition
import math
import sys
class OneEuroFilter:  
    def __init__(self, t0, x0, dx0, min_cutoff, beta, d_cutoff):
        self.t_prev = t0
        self.x_prev = x0
        self.dx_prev = dx0
        self.min_cutoff = min_cutoff
        self.beta = beta
        self.d_cutoff = d_cutoff
    def smoothing_factor(self, te, cutoff):
        r = 2.0 * math.pi * cutoff * te
        alpha = r / (r + 1.0)
        return alpha
    def exponential_smoothing(self,a, x, x_prev):
        return a * x + (1 - a) * x_prev
    def filter(self, value, time):
        dt = time - self.t_prev  
        # Compute the filtered derivative of the signal.
        alpha_d = self.smoothing_factor(dt, self.d_cutoff)
        dx = (value - self.x_prev) / (dt + sys.maxsize)
        dx_hat = self.exponential_smoothing(alpha_d, dx, self.dx_prev)       
        # The filtered signal.
        cutoff = self.min_cutoff + self.beta * abs(dx_hat)
        alpha = self.smoothing_factor(dt, cutoff)
        x_hat = self.exponential_smoothing(alpha, value, self.x_prev)
        # Memorize the previous values.
        self.x_prev = x_hat
        self.dx_prev = dx_hat
        self.t_prev = time
        return x_hat


In [170]:
# one euro filter for smoothing
import cv2
import mediapipe as mp
import time
import numpy as np
# call the OneEuroFilter class 
one_euro_filter = OneEuroFilter(0,0,dx0 = 0,min_cutoff=1.0, beta=0.701, d_cutoff=10.0)
with mp.solutions.pose.Pose(min_detection_confidence=0.5) as pose:
    #capture video
    cap = cv2.VideoCapture('prana.mp4')
    ret, frame = cap.read()
    while cap.isOpened():
        ret, frame = cap.read()
        results = pose.process(frame)
#         fps = cap.get(cv2.CAP_PROP_FPS)
#         print(fps)
        edge = np.zeros((512, 512, 3), np.uint8)
        w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        #print(length,"toatl frames")
        # this timestamp of the frame
        timestamp = cap.get(cv2.CAP_PROP_POS_MSEC)
        print(timestamp,"time for frame")        
        x=[]
        y=[]
        z=[]
        for landmark in results.pose_landmarks.landmark:
            # Extract the x, y, and z coordinates of the keypoint
            x_ = landmark.x*w
            y_ = landmark.y*h
            z_ = landmark.z
            x.append(x_)
            y.append(y_)
            z.append(z_)
            # Apply the OneEuroFilter to smooth the coordinates
        coordinates = np.array([x, y, z])
        print(x,"old...................")
        coordinates_smooth = one_euro_filter.filter(coordinates, timestamp)
        x_smooth = coordinates_smooth[0]
        y_smooth = coordinates_smooth[1]
        print(x_smooth,"new......................")
        for i in range(len(x_smooth)):
            cv2.circle(edge, (int(x_smooth[i]), int(y_smooth[i])), 3, (0, 0, 255), 2)
        cv2.imshow('Pose', edge)
        if cv2.waitKey(1) & 0xFF == ord('q'):
                break
cap.release()
cv2.destroyAllWindows()

33.333333333333336 time for frame
[185.24461948871613, 189.3456216454506, 192.5881712436676, 195.86277306079865, 179.81070965528488, 176.37461256980896, 172.9450735449791, 200.12723064422607, 167.89523375034332, 191.02640902996063, 179.49715691804886, 223.8535680770874, 145.0333382487297, 241.2351633310318, 142.0057042837143, 208.79476046562195, 168.25134128332138, 200.31660372018814, 176.21760630607605, 199.9498605132103, 171.49402397871017, 199.43069684505463, 172.51121306419373, 209.82795751094818, 155.5994508266449, 301.9436147212982, 82.91558140516281, 223.9936830997467, 156.81751984357834, 206.89618980884552, 170.33680456876755, 217.21366477012634, 165.34618091583252] old...................
[184.36434459 188.44585895 191.67300008 194.93204112 178.95625647
 175.53648758 172.12324558 199.17623418 167.09740242 190.1186593
 178.64419372 222.78982502 144.34414572 240.08882364 141.33089895
 207.80257625 167.45181775 199.36470736 175.3802274  198.9997069
 170.67909135 198.48301028 171.6

433.33333333333337 time for frame
[183.07681745290756, 187.7757889032364, 191.07537668943405, 194.43478560447693, 178.24088901281357, 174.84057086706161, 171.42401486635208, 199.37289637327194, 167.39921689033508, 189.18885737657547, 177.6035471558571, 223.69271409511566, 143.2449478507042, 243.2710418701172, 133.26001012325287, 209.13948214054108, 167.0025994181633, 199.1951390504837, 175.76958912611008, 198.69892859458923, 173.8004890680313, 200.4220529794693, 174.4157851934433, 209.1844447851181, 152.78440934419632, 290.80742275714874, 76.72670206427574, 225.14502203464508, 150.11589574813843, 209.01582276821136, 166.00656068325043, 222.8427313566208, 153.8464521765709] old...................
[183.0770155  187.77585258 191.07562582 194.43514677 178.24050827
 174.84000196 171.42333256 199.37157457 167.39757807 189.18905082
 177.60236812 223.69046567 143.24382427 243.27598819 133.26667689
 209.14042997 166.99981802 199.19641143 175.76485397 198.6998483
 173.79682803 200.4211846  174.4

700.0 time for frame
[186.67617636919022, 192.67486590147018, 195.66532945632935, 198.7181671857834, 181.86076909303665, 178.4749585390091, 175.05030781030655, 201.93600910902023, 167.87968558073044, 189.70222526788712, 179.65498596429825, 220.6968903541565, 137.66326385736465, 238.25761020183563, 133.98108220100403, 209.18773591518402, 165.6506346464157, 201.45247918367386, 173.96994596719742, 192.39285719394684, 176.65757715702057, 196.6267750263214, 175.27069252729416, 209.44546043872833, 156.54528772830963, 260.28233659267426, 87.64523780345917, 218.61754953861237, 156.64051270484924, 203.55525720119476, 173.13116759061813, 222.97326338291168, 170.85578674077988] old...................
[186.67530628 192.67333464 195.66380421 198.71664787 181.85949878
 178.47363126 175.04894185 201.93436414 167.87910847 189.7019824
 179.65410544 220.6955397  137.66704267 238.25753487 133.98211537
 209.18682497 165.65264323 201.44984929 173.97144555 192.39269848
 176.66136488 196.62631965 175.2724067

1000.0 time for frame
[188.10928201675415, 193.3505276441574, 196.32937544584274, 199.37428784370422, 183.20149320363998, 179.7905877828598, 176.36476337909698, 202.3409875035286, 170.18529158830643, 192.39876186847687, 181.21201717853546, 220.86389100551605, 144.68502295017242, 244.83123111724854, 135.93089526891708, 211.907879114151, 167.90277189016342, 202.60708504915237, 176.68664073944092, 198.56633961200714, 177.35328817367554, 201.5870525240898, 176.63260328769684, 211.57370841503143, 159.75700718164444, 258.44905614852905, 98.33709090948105, 223.77208840847015, 158.21911776065826, 213.157613158226, 172.56088250875473, 227.63841605186462, 167.20992851257324] old...................
[188.10840467 193.35046681 196.32932229 199.37424598 183.2007322
 179.7898937  176.3641155  202.34092697 170.18334122 192.39611212
 181.21072055 220.86387783 144.67971636 244.82802199 135.93164926
 211.90415635 167.90366993 202.60433924 176.68605902 198.55967371
 177.35432487 201.58131714 176.63197159 

1300.0 time for frame
[192.6514407992363, 196.69902259111404, 199.57954788208008, 202.5276744365692, 187.8521745800972, 184.47883886098862, 181.07750433683395, 205.1944819688797, 175.07170015573502, 196.83618527650833, 185.77905303239822, 222.43073511123657, 149.2410722374916, 253.36479258537292, 123.2746731042862, 252.24667954444885, 156.77138352394104, 255.49076581001282, 166.98244124650955, 254.0419909954071, 167.55533987283707, 252.4794543981552, 166.8711720108986, 212.7763260602951, 161.21655076742172, 268.0237762928009, 96.7345404624939, 230.21941220760345, 157.3065430521965, 217.56000781059265, 171.11574971675873, 241.65236246585846, 162.22162801027298] old...................
[192.64356283 196.69307138 199.57414451 202.52301339 187.84505602
 184.47139117 181.06939544 205.19526769 175.06447229 196.82844708
 185.77324673 222.43409056 149.22909613 253.37263197 123.24480442
 252.09772995 156.775028   255.27293559 166.99814915 253.81441872
 167.57014065 252.28255133 166.87788934 212.

...

4433.333333333334 time for frame
[194.18653517961502, 198.94555765390396, 202.0432733297348, 205.20423436164856, 190.55955255031586, 187.56997227668762, 184.60384130477905, 210.46200823783875, 182.27264434099197, 200.636520922184, 189.36487233638763, 231.93999564647675, 165.88357889652252, 281.1768743991852, 148.09394401311874, 268.96641886234283, 125.9164462685585, 268.9946838617325, 116.81929087638855, 261.5347809791565, 121.22186702489853, 259.7030735015869, 126.7119317650795, 225.44209492206573, 187.20860135555267, 235.06039834022522, 208.12685465812683, 215.20859217643738, 253.9960845708847, 208.89572072029114, 258.04855465888977, 217.77185010910034, 274.5062619447708] old...................
[194.18702892 198.94594782 202.04362331 205.20455965 190.56005854
 187.57052274 184.6044567  210.46201339 182.27315786 200.63674546
 189.36530959 231.9398117  165.88395074 281.17695941 148.08900497
 268.96706991 125.91636876 268.99528441 116.82024911 261.53557378
 121.22252944 259.70490564 126

4733.333333333333 time for frame
[195.51980584859848, 200.01915574073792, 203.06299126148224, 206.16488134860992, 191.76370686292648, 188.81784296035767, 185.89400058984756, 211.54902493953705, 183.78138548135757, 202.20751523971558, 190.93970227241516, 231.81389212608337, 170.11449599266052, 282.5533411502838, 151.99165838956833, 271.9589272737503, 127.95929425954819, 272.88787293434143, 117.54892957210541, 265.7649236917496, 122.03830927610397, 263.9759283065796, 127.69898039102554, 226.54196155071259, 190.35405051708221, 234.7647774219513, 194.47553753852844, 219.37591302394867, 247.29706192016602, 215.38640999794006, 254.8301682472229, 213.76003515720367, 265.08685410022736] old...................
[195.51919774 200.01875161 203.06257683 206.16445149 191.76334305
 188.81745839 185.89358608 211.54799875 183.7802415  202.20607302
 190.93834872 231.8137154  170.11232618 282.55381083 151.99592723
 271.95706323 127.96069516 272.88492621 117.55222073 265.76196596
 122.04105721 263.9731181

5000.0 time for frame
[198.62318420410156, 203.1547315120697, 206.5172984600067, 209.93191850185394, 194.16582041978836, 190.80407625436783, 187.50036245584488, 214.941792845726, 184.6432985663414, 205.77045392990112, 193.15638726949692, 234.32850909233093, 169.64831709861755, 283.6272659301758, 136.74194103479385, 273.6720572710037, 134.77390575408936, 273.53145825862885, 131.1513637304306, 266.755868434906, 136.73282992839813, 265.2333335876465, 139.7512801885605, 232.39736592769623, 196.4711481332779, 235.09693956375122, 215.6429487466812, 221.42047917842865, 256.8194385766983, 219.36398267745972, 262.50810849666595, 215.15184438228607, 272.5640354156494] old...................
[198.62287704 203.1543132  206.51679663 209.93133051 194.16571029
 190.80394563 187.50008171 214.94175306 184.64274607 205.77027514
 193.15624783 234.32891308 169.6486145  283.6278714  136.74111932
 273.67367995 134.80042743 273.53266847 131.19810233 266.75684173
 136.78154503 265.23416602 139.79214872 232.39

5300.0 time for frame
[198.39562433958054, 203.2607010602951, 206.39451026916504, 209.5844622850418, 194.80655020475388, 191.6287221312523, 188.46875327825546, 214.14081406593323, 185.33945727348328, 204.4756362438202, 192.44481348991394, 231.6093338727951, 167.6769301891327, 281.3671307563782, 140.39467191696167, 275.9671607017517, 125.52781462669373, 276.94665908813477, 119.76239603757858, 270.22162199020386, 125.33448493480682, 268.3714987039566, 129.1884708404541, 230.90449965000153, 194.66197037696838, 239.69225251674652, 213.10260772705078, 220.35449182987213, 242.5772910118103, 215.6943484544754, 245.68114125728607, 216.63609564304352, 250.72903299331665] old...................
[198.39446497 203.25973075 206.39365687 209.58372537 194.80542404
 191.62764214 188.46773689 214.14131611 185.33953046 204.47547241
 192.44466091 231.61002776 167.67735205 281.36661102 140.39470494
 275.96694416 125.52800745 276.94824674 119.76324019 270.22288879
 125.33505784 268.37202244 129.1890933  23

5633.333333333333 time for frame
[198.15614622831345, 202.88452005386353, 205.89406490325928, 208.95394468307495, 194.5111107826233, 191.37914073467255, 188.27282214164734, 212.51748836040497, 184.4071478843689, 203.25970888137817, 191.82080554962158, 226.7115273475647, 166.87621760368347, 275.95888447761536, 147.7971857190132, 276.198846578598, 125.78976196050644, 278.8979604244232, 117.69125884771347, 272.53855335712433, 121.91092318296432, 270.16385781764984, 125.73150169849396, 228.21772754192352, 193.7727239727974, 241.70206820964813, 212.6875865459442, 217.51018106937408, 243.3958386182785, 211.37079572677612, 245.94898116588593, 215.4320502281189, 255.8181848526001] old...................
[198.16027926 202.88698092 205.89599008 208.9553459  194.51491668
 191.38317795 188.27707335 212.51682037 184.4105392  203.26250197
 191.8247892  226.71409141 166.87257088 275.96027331 147.78776324
 276.20297993 125.7929716  278.90250765 117.69935455 272.54106965
 121.91866584 270.16527996 125.

In [175]:
# one euro filter for one point visulization
import cv2
import mediapipe as mp
import time
import numpy as np
one_euro_filter = OneEuroFilter(0,0,dx0 = 0,min_cutoff=1.0, beta=0.701, d_cutoff=10.0)
data=['LEFT_ANKLE','LEFT_EAR','LEFT_ELBOW','LEFT_EYE','LEFT_EYE_INNER','LEFT_EYE_OUTER','LEFT_FOOT_INDEX','LEFT_HEEL','LEFT_HIP','LEFT_INDEX','LEFT_KNEE','LEFT_MOUTH','LEFT_PINKY','LEFT_SHOULDER','LEFT_THUMB','LEFT_WRIST','NOSE','RIGHT_ANKLE','RIGHT_EAR','RIGHT_ELBOW','RIGHT_EYE','RIGHT_EYE_INNER','RIGHT_EYE_OUTER','RIGHT_FOOT_INDEX','RIGHT_HEEL','RIGHT_HIP','RIGHT_INDEX','RIGHT_KNEE','RIGHT_MOUTH','RIGHT_PINKY','RIGHT_SHOULDER','RIGHT_THUMB','RIGHT_WRIST']
with mp.solutions.pose.Pose(min_detection_confidence=0.5) as pose:
    cap = cv2.VideoCapture('prana.mp4')
    ret, frame = cap.read()
    while cap.isOpened():
        ret, frame = cap.read()
        results = pose.process(frame)
#         fps = cap.get(cv2.CAP_PROP_FPS)
#         print(fps)
        edge = np.zeros((512, 512, 3), np.uint8)
        w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        print(length,"toatl frames")
        # this timestamp of the frame
        timestamp = cap.get(cv2.CAP_PROP_POS_MSEC)
        print(timestamp,"time for frame")        
        for i in range(len(data)):
        # Extract the x, y, and z coordinates of the keypoints
            x = results.pose_landmarks.landmark[i].x*w
            y = results.pose_landmarks.landmark[i].y*h
            z = results.pose_landmarks.landmark[i].z
            coordinates = np.array([x, y, z])
            #print(coordinates)
            print(coordinates,"old")
            coordinates_smooth = one_euro_filter.filter(coordinates,timestamp)
            #print(coordinates_smooth,">>>>>>>>>>>>>>",i)
            x_smooth = coordinates_smooth[0]
            y_smooth = coordinates_smooth[1]
            print(x_smooth, y_smooth,"new")
            cv2.circle(edge, (int(x_smooth), int(y_smooth)), 3, (0, 0, 255), 2)
            #visualize_pose(edge,int(x_smooth), int(y_smooth))
        cv2.imshow('Pose', edge)
        if cv2.waitKey(1) & 0xFF == ord('q'):
                break
cap.release()
cv2.destroyAllWindows()

1800 toatl frames
33.333333333333336 time for frame
[ 1.85244619e+02  2.84856284e+02 -1.90108389e-01] old
184.3643445855953 283.5026581818235 new
[ 1.89345622e+02  2.78383749e+02 -1.60602137e-01] old
184.3643445855953 283.5026581818235 new
[ 1.92588171e+02  2.78457499e+02 -1.60732210e-01] old
184.3643445855953 283.5026581818235 new
[ 1.95862773e+02  2.78469901e+02 -1.60934657e-01] old
184.3643445855953 283.5026581818235 new
[ 1.79810710e+02  2.78471639e+02 -1.66319504e-01] old
184.3643445855953 283.5026581818235 new
[ 1.76374613e+02  2.78485544e+02 -1.66252494e-01] old
184.3643445855953 283.5026581818235 new
[ 1.72945074e+02  2.78547428e+02 -1.66316256e-01] old
184.3643445855953 283.5026581818235 new
[2.00127231e+02 2.82813535e+02 1.48171745e-02] old
184.3643445855953 283.5026581818235 new
[ 1.67895234e+02  2.82764161e+02 -7.34946644e-03] old
184.3643445855953 283.5026581818235 new
[ 1.91026409e+02  2.93815613e+02 -1.25550404e-01] old
184.3643445855953 283.5026581818235 new
[ 1.7949715

1800 toatl frames
200.0 time for frame
[ 1.85244922e+02  2.77855310e+02 -2.55230516e-01] old
185.24782894839925 277.86004827525005 new
[ 1.89549672e+02  2.71451633e+02 -2.25862414e-01] old
185.24782894839925 277.86004827525005 new
[ 1.92815090e+02  2.71664085e+02 -2.25941211e-01] old
185.24782894839925 277.86004827525005 new
[ 1.96109572e+02  2.71810191e+02 -2.26108104e-01] old
185.24782894839925 277.86004827525005 new
[ 1.80004221e+02  2.71319175e+02 -2.31199637e-01] old
185.24782894839925 277.86004827525005 new
[ 1.76500051e+02  2.71263943e+02 -2.31168076e-01] old
185.24782894839925 277.86004827525005 new
[ 1.73002705e+02  2.71260982e+02 -2.31189683e-01] old
185.24782894839925 277.86004827525005 new
[ 1.99601703e+02  2.76757622e+02 -4.58837561e-02] old
185.24782894839925 277.86004827525005 new
[ 1.67683186e+02  2.75828333e+02 -6.26237765e-02] old
185.24782894839925 277.86004827525005 new
[ 1.90569765e+02  2.87573211e+02 -1.88684121e-01] old
185.24782894839925 277.86004827525005 new
[

1800 toatl frames
300.0 time for frame
[ 1.84897151e+02  2.75019529e+02 -1.97487891e-01] old
184.8962123681651 275.0214193514143 new
[ 1.89098255e+02  2.68577485e+02 -1.63444906e-01] old
184.8962123681651 275.0214193514143 new
[ 1.92345608e+02  2.68755434e+02 -1.63474768e-01] old
184.8962123681651 275.0214193514143 new
[ 1.95629914e+02  2.68867421e+02 -1.63576856e-01] old
184.8962123681651 275.0214193514143 new
[ 1.79545822e+02  2.68474209e+02 -1.67401388e-01] old
184.8962123681651 275.0214193514143 new
[ 1.76071490e+02  2.68423698e+02 -1.67389065e-01] old
184.8962123681651 275.0214193514143 new
[ 1.72598283e+02  2.68418763e+02 -1.67370141e-01] old
184.8962123681651 275.0214193514143 new
[1.99228849e+02 2.73918664e+02 3.00722867e-02] old
184.8962123681651 275.0214193514143 new
[1.67567645e+02 2.73169341e+02 1.86174214e-02] old
184.8962123681651 275.0214193514143 new
[ 1.90392697e+02  2.84834161e+02 -1.26427263e-01] old
184.8962123681651 275.0214193514143 new
[ 1.78722483e+02  2.8454364

1800 toatl frames
400.0 time for frame
[ 1.83117110e+02  2.73746252e+02 -1.40312478e-01] old
183.1184948131165 273.74924176928727 new
[ 1.87788518e+02  2.66971035e+02 -1.06231332e-01] old
183.1184948131165 273.74924176928727 new
[ 1.91127164e+02  2.67061887e+02 -1.06318347e-01] old
183.1184948131165 273.74924176928727 new
[ 1.94510191e+02  2.67092829e+02 -1.06359832e-01] old
183.1184948131165 273.74924176928727 new
[ 1.78160220e+02  2.66965156e+02 -1.05262965e-01] old
183.1184948131165 273.74924176928727 new
[ 1.74720312e+02  2.66919580e+02 -1.05302267e-01] old
183.1184948131165 273.74924176928727 new
[ 1.71279895e+02  2.66919923e+02 -1.05308630e-01] old
183.1184948131165 273.74924176928727 new
[1.99094348e+02 2.72213166e+02 8.74152631e-02] old
183.1184948131165 273.74924176928727 new
[1.67054205e+02 2.71734896e+02 9.69893038e-02] old
183.1184948131165 273.74924176928727 new
[ 1.89228169e+02  2.83306739e+02 -6.99668974e-02] old
183.1184948131165 273.74924176928727 new
[ 1.77353603e+02 

1800 toatl frames
566.6666666666666 time for frame
[ 1.85116314e+02  2.72815890e+02 -1.94518402e-01] old
185.11449214644603 272.81612159080237 new
[ 1.90666768e+02  2.65461552e+02 -1.69287965e-01] old
185.11449214644603 272.81612159080237 new
[ 1.93792882e+02  2.65291200e+02 -1.69237807e-01] old
185.11449214644603 272.81612159080237 new
[ 1.96975284e+02  2.65066903e+02 -1.69331312e-01] old
185.11449214644603 272.81612159080237 new
[ 1.80126223e+02  2.65571480e+02 -1.73115715e-01] old
185.11449214644603 272.81612159080237 new
[ 1.76705807e+02  2.65293453e+02 -1.73200980e-01] old
185.11449214644603 272.81612159080237 new
[ 1.73259631e+02  2.65078726e+02 -1.73116013e-01] old
185.11449214644603 272.81612159080237 new
[2.00744184e+02 2.67865605e+02 7.49314530e-03] old
185.11449214644603 272.81612159080237 new
[1.67234600e+02 2.67216103e+02 5.57430973e-03] old
185.11449214644603 272.81612159080237 new
[ 1.89185094e+02  2.82294517e+02 -1.28305346e-01] old
185.11449214644603 272.81612159080237

1800 toatl frames
666.6666666666666 time for frame
[ 1.86494741e+02  2.72358520e+02 -1.16962619e-01] old
186.49307568719115 272.35696014537973 new
[ 1.92354888e+02  2.64604039e+02 -9.06024203e-02] old
186.49307568719115 272.35696014537973 new
[ 1.95346175e+02  2.64363005e+02 -9.05197486e-02] old
186.49307568719115 272.35696014537973 new
[ 1.98399859e+02  2.64038565e+02 -9.06694978e-02] old
186.49307568719115 272.35696014537973 new
[ 1.81595022e+02  2.64652491e+02 -9.77553278e-02] old
186.49307568719115 272.35696014537973 new
[ 1.78197354e+02  2.64233422e+02 -9.78391394e-02] old
186.49307568719115 272.35696014537973 new
[ 1.74764669e+02  2.63879972e+02 -9.77535471e-02] old
186.49307568719115 272.35696014537973 new
[2.01590138e+02 2.66044643e+02 8.57367590e-02] old
186.49307568719115 272.35696014537973 new
[1.67758531e+02 2.65091858e+02 7.43379593e-02] old
186.49307568719115 272.35696014537973 new
[ 1.89651370e+02  2.82010889e+02 -5.14411032e-02] old
186.49307568719115 272.35696014537973

1800 toatl frames
766.6666666666666 time for frame
[ 1.86826540e+02  2.72688453e+02 -1.20451093e-01] old
186.8270809478942 272.68719768409267 new
[ 1.92654042e+02  2.64825675e+02 -9.26600471e-02] old
186.8270809478942 272.68719768409267 new
[ 1.95637113e+02  2.64572175e+02 -9.25579444e-02] old
186.8270809478942 272.68719768409267 new
[ 1.98678069e+02  2.64229088e+02 -9.26031396e-02] old
186.8270809478942 272.68719768409267 new
[ 1.81984755e+02  2.64848936e+02 -9.73217413e-02] old
186.8270809478942 272.68719768409267 new
[ 1.78621281e+02  2.64393711e+02 -9.73829180e-02] old
186.8270809478942 272.68719768409267 new
[ 1.75223927e+02  2.64010863e+02 -9.72524732e-02] old
186.8270809478942 272.68719768409267 new
[2.01894567e+02 2.66275055e+02 8.61811861e-02] old
186.8270809478942 272.68719768409267 new
[1.68168833e+02 2.65341454e+02 7.95018151e-02] old
186.8270809478942 272.68719768409267 new
[ 1.89906602e+02  2.82358804e+02 -5.49700595e-02] old
186.8270809478942 272.68719768409267 new
[ 1.7

In [176]:
######## Moving Averages for smoothing
def visualize_pose(edge, x_smooth, y_smooth, z_smooth):
    for line in LINES_BODY:
        # Extract the coordinates of the two body parts
        x1 = results.pose_landmarks.landmark[line[0]].x*w
        y1 = results.pose_landmarks.landmark[line[0]].y*h
        z1 = results.pose_landmarks.landmark[line[0]].z
        x2 = results.pose_landmarks.landmark[line[1]].x*w
        y2 = results.pose_landmarks.landmark[line[1]].y*h
        z2 = results.pose_landmarks.landmark[line[1]].z
        cv2.line(edge, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255,0), 2)    
    return edge
import mediapipe as mp
import numpy as np
filter_length = 2
LINES_BODY = [[9,10],[4,6],[1,3],
              [12,14],[14,16],[16,20],[20,18],[18,16],
              [12,11],[11,23],[23,24],[24,12],
              [11,13],[13,15],[15,19],[19,17],[17,15],
              [24,26],[26,28],[32,30],
              [23,25],[25,27],[29,31]]

with mp.solutions.pose.Pose(min_detection_confidence=0.5) as pose:
    cap = cv2.VideoCapture('prana.mp4')
    w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    x_filter = np.zeros(filter_length)
    y_filter = np.zeros(filter_length)
    z_filter = np.zeros(filter_length)
    # Process each frame of the video
    while cap.isOpened():
        ret, frame = cap.read()
        edge = np.zeros((512, 512, 3), np.uint8)
        results = pose.process(frame)
        for i in range(len(results.pose_landmarks.landmark)):
            x = (results.pose_landmarks.landmark[i].x)*w
            y = (results.pose_landmarks.landmark[i].y)*h
            z = results.pose_landmarks.landmark[i].z
            print(x)
        # moving average filter for each coordinate
            x_filter = np.roll(x_filter, -1)
            x_filter[-1] = x
            x_smooth = np.mean(x_filter)
            print(x_filter,"smooth")
            y_filter = np.roll(y_filter, -1)
            y_filter[-1] = y
            y_smooth = np.mean(y_filter)
            z_filter = np.roll(z_filter, -1)
            z_filter[-1] = z
            z_smooth = np.mean(z_filter)
            cv2.circle(edge, (int(x_smooth), int(y_smooth)), 2, (255,255,0), 2)
            #visualize_pose(edge,x_smooth, y_smooth,z_smooth)
        cv2.imshow('Pose', edge)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

186.64802026748657
[  0.         186.64802027] smooth
190.49062031507492
[186.64802027 190.49062032] smooth
193.80942249298096
[190.49062032 193.80942249] smooth
197.16564917564392
[193.80942249 197.16564918] smooth
180.75727254152298
[197.16564918 180.75727254] smooth
177.18608182668686
[180.75727254 177.18608183] smooth
173.62002140283585
[177.18608183 173.6200214 ] smooth
201.17794811725616
[173.6200214  201.17794812] smooth
168.34025019407272
[201.17794812 168.34025019] smooth
192.55044424533844
[168.34025019 192.55044425] smooth
180.70912766456604
[192.55044425 180.70912766] smooth
225.35812270641327
[180.70912766 225.35812271] smooth
146.42021077871323
[225.35812271 146.42021078] smooth
242.67897725105286
[146.42021078 242.67897725] smooth
141.83135908842087
[242.67897725 141.83135909] smooth
207.1014256477356
[141.83135909 207.10142565] smooth
167.36165928840637
[207.10142565 167.36165929] smooth
197.37548291683197
[167.36165929 197.37548292] smooth
174.71435844898224
[197.37548

183.88198274374008
[158.65551919 183.88198274] smooth
188.4293202161789
[183.88198274 188.42932022] smooth
191.7218901515007
[188.42932022 191.72189015] smooth
195.0415514111519
[191.72189015 195.04155141] smooth
178.8037085533142
[195.04155141 178.80370855] smooth
175.32220113277435
[178.80370855 175.32220113] smooth
171.8507364988327
[175.32220113 171.8507365 ] smooth
199.0675714612007
[171.8507365  199.06757146] smooth
166.7962383031845
[199.06757146 166.7962383 ] smooth
189.34967505931854
[166.7962383  189.34967506] smooth
177.68840265274048
[189.34967506 177.68840265] smooth
223.49880361557007
[177.68840265 223.49880362] smooth
141.82313126325607
[223.49880362 141.82313126] smooth
243.070960521698
[141.82313126 243.07096052] smooth
136.42769861221313
[243.07096052 136.42769861] smooth
207.38630199432373
[136.42769861 207.38630199] smooth
165.54104727506638
[207.38630199 165.54104728] smooth
197.613545358181
[165.54104728 197.61354536] smooth
173.861024081707
[197.61354536 173.8610

[184.35674036 188.68964618] smooth
191.976771235466
[188.68964618 191.97677124] smooth
195.30320835113525
[191.97677124 195.30320835] smooth
179.1505234837532
[195.30320835 179.15052348] smooth
175.67940974235535
[179.15052348 175.67940974] smooth
172.21109104156494
[175.67940974 172.21109104] smooth
199.2094167470932
[172.21109104 199.20941675] smooth
167.42942994832993
[199.20941675 167.42942995] smooth
189.98843258619308
[167.42942995 189.98843259] smooth
178.28723102808
[189.98843259 178.28723103] smooth
222.81260299682617
[178.28723103 222.812603  ] smooth
142.84530544281006
[222.812603   142.84530544] smooth
243.8473768234253
[142.84530544 243.84737682] smooth
134.8831422328949
[243.84737682 134.88314223] smooth
210.36128997802734
[134.88314223 210.36128998] smooth
167.05520910024643
[210.36128998 167.0552091 ] smooth
200.2828333377838
[167.0552091  200.28283334] smooth
175.74139672517776
[200.28283334 175.74139673] smooth
200.28403121232986
[175.74139673 200.28403121] smooth
172

[183.96973008 189.6206851 ] smooth
192.7734062075615
[189.6206851  192.77340621] smooth
195.98891288042068
[192.77340621 195.98891288] smooth
179.3066222667694
[195.98891288 179.30662227] smooth
175.95829671621323
[179.30662227 175.95829672] smooth
172.58121007680893
[175.95829672 172.58121008] smooth
200.17440754175186
[172.58121008 200.17440754] smooth
167.5540573000908
[200.17440754 167.5540573 ] smooth
189.3153359889984
[167.5540573  189.31533599] smooth
178.09828144311905
[189.31533599 178.09828144] smooth
221.4791145324707
[178.09828144 221.47911453] smooth
142.05485343933105
[221.47911453 142.05485344] smooth
239.68927597999573
[142.05485344 239.68927598] smooth
134.04878026247025
[239.68927598 134.04878026] smooth
210.38711082935333
[134.04878026 210.38711083] smooth
165.80485796928406
[210.38711083 165.80485797] smooth
202.97298127412796
[165.80485797 202.97298127] smooth
174.8370619416237
[202.97298127 174.83706194] smooth
198.7244227528572
[174.83706194 198.72442275] smooth


In [177]:
# Kalman filter for smoothing
import cv2
import numpy as np
import mediapipe as mp
from pykalman import KalmanFilter
num_keypoints=33
LINES_BODY = [[9,10],[4,6],[1,3],
              [12,14],[14,16],[16,20],[20,18],[18,16],
              [12,11],[11,23],[23,24],[24,12],
              [11,13],[13,15],[15,19],[19,17],[17,15],
              [24,26],[26,28],[32,30],
              [23,25],[25,27],[29,31]]
def visualize_pose(edge, x_smooth, y_smooth):
    for line in LINES_BODY:
        # coordinates of the two body parts
        x1 = results.pose_landmarks.landmark[line[0]].x*w
        y1 = results.pose_landmarks.landmark[line[0]].y*h
#         z1 = results.pose_landmarks.landmark[line[0]].z
        x2 = results.pose_landmarks.landmark[line[1]].x*w
        y2 = results.pose_landmarks.landmark[line[1]].y*h
#         z2 = results.pose_landmarks.landmark[line[1]].z
        edge =cv2.line(edge, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0,255), 2)    
        #edge =cv2.line(edge, (0, 0), (100, 100), (0, 0,255), 2)
    return edge
kf = KalmanFilter(transition_matrices=np.eye(3), observation_matrices=np.eye(3,3))
with mp.solutions.pose.Pose(min_detection_confidence=0.5) as pose:
    cap = cv2.VideoCapture('prana.mp4')
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    while cap.isOpened():
        ret, frame = cap.read()
        results = pose.process(frame)
        edge = np.zeros((512, 512, 3), np.uint8)
        for i in range(len(results.pose_landmarks.landmark)):
    # Extract the x, y, and z coordinates of the keypoints
            x = results.pose_landmarks.landmark[i].x*w
            y = results.pose_landmarks.landmark[i].y*h
            z = results.pose_landmarks.landmark[i].z
            print(x)
            coordinates = np.array([x, y,z])
            coordinates_smooth = kf.filter_update(filtered_state_mean=coordinates,
                                                  filtered_state_covariance=np.eye(3),
                                                  observation=coordinates)[0]
                                                  #observation_covariance=np.eye(3))[0]
            print(coordinates_smooth,"meow")           
            x_smooth = coordinates_smooth[0]
            y_smooth = coordinates_smooth[1]
            print(x_smooth,">>>>>>>>>>>>")
#             cap.set(cv2.CAP_PROP_FPS, 33)
            # Draw circles at the keypoint locations
            cv2.circle(edge, (int(x_smooth), int(y_smooth)), 3, (0, 0, 255), 2)
            #visualize_pose(edge,int(x_smooth), int(y_smooth))
        cv2.imshow('Pose', edge)
        if cv2.waitKey(1) & 0xFF == ord('q'):
                break
cap.release()
cv2.destroyAllWindows()

186.64802026748657
[ 1.86648020e+02  2.85435812e+02 -2.35337362e-01] meow
186.64802026748657 >>>>>>>>>>>>
190.49062031507492
[ 1.90490620e+02  2.78445547e+02 -2.08703145e-01] meow
190.49062031507492 >>>>>>>>>>>>
193.80942249298096
[ 1.93809422e+02  2.78401601e+02 -2.08814234e-01] meow
193.80942249298096 >>>>>>>>>>>>
197.16564917564392
[ 1.97165649e+02  2.78302188e+02 -2.08993807e-01] meow
197.16564917564392 >>>>>>>>>>>>
180.75727254152298
[ 1.80757273e+02  2.78799620e+02 -2.14396954e-01] meow
180.75727254152298 >>>>>>>>>>>>
177.18608182668686
[ 1.77186082e+02  2.78903646e+02 -2.14351594e-01] meow
177.18608182668686 >>>>>>>>>>>>
173.62002140283585
[ 1.73620021e+02  2.79056983e+02 -2.14411944e-01] meow
173.62002140283585 >>>>>>>>>>>>
201.17794811725616
[ 2.01177948e+02  2.82723413e+02 -3.27122919e-02] meow
201.17794811725616 >>>>>>>>>>>>
168.34025019407272
[ 1.68340250e+02  2.83507133e+02 -5.29015325e-02] meow
168.34025019407272 >>>>>>>>>>>>
192.55044424533844
[ 1.92550444e+02  2.9453133

184.96423214673996
[ 1.84964232e+02  2.80739801e+02 -2.37937436e-01] meow
184.96423214673996 >>>>>>>>>>>>
189.72262543439865
[ 1.89722625e+02  2.74320738e+02 -2.09025323e-01] meow
189.72262543439865 >>>>>>>>>>>>
193.07393962144852
[ 1.93073940e+02  2.74597499e+02 -2.09130049e-01] meow
193.07393962144852 >>>>>>>>>>>>
196.4566526412964
[ 1.96456653e+02  2.74813771e+02 -2.09319249e-01] meow
196.4566526412964 >>>>>>>>>>>>
179.9680426120758
[ 1.79968043e+02  2.73952847e+02 -2.11953923e-01] meow
179.9680426120758 >>>>>>>>>>>>
176.4596374630928
[ 1.76459637e+02  2.73808930e+02 -2.11965904e-01] meow
176.4596374630928 >>>>>>>>>>>>
172.9622067809105
[ 1.72962207e+02  2.73723292e+02 -2.12030947e-01] meow
172.9622067809105 >>>>>>>>>>>>
200.84986358880997
[ 2.00849864e+02  2.79783711e+02 -4.62212153e-02] meow
200.84986358880997 >>>>>>>>>>>>
168.0136902332306
[ 1.68013690e+02  2.78233867e+02 -5.09696826e-02] meow
168.0136902332306 >>>>>>>>>>>>
190.70125263929367
[ 1.90701253e+02  2.90701203e+02 -1.7

184.6475334763527
[ 1.84647533e+02  2.77163858e+02 -2.32632965e-01] meow
184.6475334763527 >>>>>>>>>>>>
189.06181007623672
[ 1.89061810e+02  2.70786917e+02 -2.00244412e-01] meow
189.06181007623672 >>>>>>>>>>>>
192.34110659360886
[ 1.92341107e+02  2.71045847e+02 -2.00322405e-01] meow
192.34110659360886 >>>>>>>>>>>>
195.6530482172966
[ 1.95653048e+02  2.71237271e+02 -2.00461671e-01] meow
195.6530482172966 >>>>>>>>>>>>
179.4426475763321
[ 1.79442648e+02  2.70503612e+02 -2.05321357e-01] meow
179.4426475763321 >>>>>>>>>>>>
175.94613647460938
[ 1.75946136e+02  2.70392096e+02 -2.05267057e-01] meow
175.94613647460938 >>>>>>>>>>>>
172.45977705717087
[ 1.72459777e+02  2.70329461e+02 -2.05272585e-01] meow
172.45977705717087 >>>>>>>>>>>>
199.31103038787842
[ 1.99311030e+02  2.76276648e+02 -9.38999373e-03] meow
199.31103038787842 >>>>>>>>>>>>
167.24602204561234
[ 1.67246022e+02  2.74877930e+02 -2.53002569e-02] meow
167.24602204561234 >>>>>>>>>>>>
190.02671617269516
[ 1.90026716e+02  2.87050867e+02 

184.35674035549164
[ 1.84356740e+02  2.74941680e+02 -1.96315065e-01] meow
184.35674035549164 >>>>>>>>>>>>
188.68964618444443
[ 1.88689646e+02  2.68156765e+02 -1.66490495e-01] meow
188.68964618444443 >>>>>>>>>>>>
191.976771235466
[ 1.91976771e+02  2.68307397e+02 -1.66506991e-01] meow
191.976771235466 >>>>>>>>>>>>
195.30320835113525
[ 1.95303208e+02  2.68396297e+02 -1.66607276e-01] meow
195.30320835113525 >>>>>>>>>>>>
179.1505234837532
[ 1.79150523e+02  2.68048124e+02 -1.69612363e-01] meow
179.1505234837532 >>>>>>>>>>>>
175.67940974235535
[ 1.75679410e+02  2.67977593e+02 -1.69625267e-01] meow
175.67940974235535 >>>>>>>>>>>>
172.21109104156494
[ 1.72211091e+02  2.67950621e+02 -1.69610560e-01] meow
172.21109104156494 >>>>>>>>>>>>
199.2094167470932
[1.99209417e+02 2.73330746e+02 1.75918136e-02] meow
199.2094167470932 >>>>>>>>>>>>
167.42942994832993
[1.67429430e+02 2.72549686e+02 8.84687342e-03] meow
167.42942994832993 >>>>>>>>>>>>
189.98843258619308
[ 1.89988433e+02  2.84598641e+02 -1.27925